# Holder Wallet Conformance Flows (v3) - CTWalletSameDeferred

# 0.0 Initial setup

## 0.1 Setup conformance

In [1]:
import uuid
import asyncio
from rich.console import Console

console = Console()

loop = asyncio.get_event_loop()

## 0.2 Create did:key:jwk_jcs-pub identifier using ES256 algorithm

In [2]:
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

key_did = KeyDid()

# generate keypair
key_did.create_keypair()

# create public key jwk
public_key_jwk = PublicKeyJWK(
    kty=key_did.public_key_jwk['kty'],
    crv=key_did.public_key_jwk['crv'],
    x=key_did.public_key_jwk['x'],
    y=key_did.public_key_jwk['y']
)

# generate did
key_did.generate_did(public_key_jwk)

print("Decentralised identifier: ", key_did._did)

Decentralised identifier:  did:key:z2dmzD81cgPx8Vki7JbuuMmFYrWPgYoytykUZ3eyqht1j9KbrqbZd74bTU8HuiECBaqGrab7Qpu9QMUmNWdJ8GXkq1p5JpWCRYgogFxZLDBkCA1t2KcUJhLuptvoGKygDxp1MpgEcsiUQtHjNzLnxSg55ixd81g6waogrfe1tf2SLDt2P6


## 1.1 Initiate Credential Issuance

In [3]:
from ebsi_wallet.util import parse_query_string_parameters_from_url
from ebsi_wallet.siop_auth.util import (
    accept_and_fetch_credential_offer, 
    fetch_openid_credential_issuer_configuration,
    fetch_openid_auth_server_configuration,
    fetch_credential_offer,
    CredentialTypes
)

credential_type = CredentialTypes.CTWalletSameDeferred
qr_code_data = await fetch_credential_offer(key_did._did, credential_type)
qr_code_data = qr_code_data.replace("openid-credential-offer://", "")
credential_offer_uri = parse_query_string_parameters_from_url(
    qr_code_data).get("credential_offer_uri")[0]
console.log("Credential offer URI: ", credential_offer_uri)

credential_offer = await accept_and_fetch_credential_offer(credential_offer_uri)
console.log("Credential offer: ", credential_offer)

credential_issuer_configuration = await fetch_openid_credential_issuer_configuration(credential_offer.credential_issuer)
console.log("Credential issuer configuration: ", credential_issuer_configuration)

auth_server_configuration = await fetch_openid_auth_server_configuration(credential_issuer_configuration.authorization_server)
console.log("Authorization server configuration: ", auth_server_configuration)

[21:12:18] Credential offer URI:                                                                   ]8;id=58378;file:///tmp/ipykernel_54329/1703696794.py\1703696794.py]8;;\:]8;id=486171;file:///tmp/ipykernel_54329/1703696794.py#15\15]8;;\
           https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/offers/94299df7-8522-41f5-b7                 
           4d-6c366ec155f3                                                                                         

[21:12:19] Credential offer:                                                                       ]8;id=171698;file:///tmp/ipykernel_54329/1703696794.py\1703696794.py]8;;\:]8;id=330760;file:///tmp/ipykernel_54329/1703696794.py#18\18]8;;\
           CredentialOffer(                                                                                        
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               credentials=[                                                                                       
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'CTWalletSameDeferred'                                                                  
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       }                                                                                           
                   }                                                                                               
               ],                                                                                                  
               grants={                                                                                            
                   'authorization_code': {                                                                         
                       'issuer_state':                                                                             
           'eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6InVPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNF                 
           ZqcFB0cDY1dTVjWXcifQ.eyJjbGllbnRfaWQiOiJkaWQ6a2V5OnoyZG16RDgxY2dQeDhWa2k3SmJ1dU1tRllyV1                 
           BnWW95dHlrVVozZXlxaHQxajlLYnJxYlpkNzRiVFU4SHVpRUNCYXFHcmFiN1FwdTlRTVVtTldkSjhHWGtxMXA1S                 
           nBXQ1JZZ29nRnhaTERCa0NBMXQyS2NVSmhMdXB0dm9HS3lnRHhwMU1wZ0Vjc2lVUXRIak56TG54U2c1NWl4ZDgx                 
           ZzZ3YW9ncmZlMXRmMlNMRHQyUDYiLCJjcmVkZW50aWFsX3R5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiw                 
           iVmVyaWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lRGVmZXJyZWQiXSwiaWF0IjoxNjg3MTg5MzM4LC                 
           JleHAiOjE2ODcxODk2MzgsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY                 
           2UvdjMvaXNzdWVyLW1vY2siLCJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2NvbmZvcm1h                 
           bmNlL3YzL2F1dGgtbW9jayIsInN1YiI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l                 
           0eWtVWjNleXFodDFqOUticnFiWmQ3NGJUVThIdWlFQ0JhcUdyYWI3UXB1OVFNVW1OV2RKOEdYa3ExcDVKcFdDUl                 
           lnb2dGeFpMREJrQ0ExdDJLY1VKaEx1cHR2b0dLeWdEeHAxTXBnRWNzaVVRdEhqTnpMbnhTZzU1aXhkODFnNndhb                 
           2dyZmUxdGYyU0xEdDJQNiJ9.MWAsaWARWteEqBmt2Lsf86p-MobXXrOJrSsqLRzpT6ePnZBfQotXNq6pyhrdPkO                 
           Lf5W6EMUCMEP0epdCEupcUg'      

[21:12:20] Credential issuer configuration:                                                        ]8;id=722552;file:///tmp/ipykernel_54329/1703696794.py\1703696794.py]8;;\:]8;id=606286;file:///tmp/ipykernel_54329/1703696794.py#21\21]8;;\
           OpenIDCredentialIssuerConfig(                                                                           
               credential_issuer='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock',                     
               authorization_server='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                    
               credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer-mock/cre                 
           dential',                                                                                               
               deferred_credential_endpoint='https://api-conformance.ebsi.eu/conformance/v3/issuer                 
           -mock/credential_deferred',                                                                             
               credentials_supported=[                                                                             
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAuthorisationToOnboard'                                                      
                       ],                                                                                          
                       'trust_framework': {                                                                        
                           'name': 'ebsi',                                                                         
                           'type': 'Accreditation',                                                                
                           'uri': 'TIR link towards accreditation'                                                 
                       },                                                                                          
                       'display': [                                                                                
                           {'name': 'Verifiable Authorisation to onboard', 'locale': 'en-GB'}                      
                       ]                                                                                           
                   },                                                                                              
                   {                                                                                               
                       'format': 'jwt_vc',                                                                         
                       'types': [                                                                                  
                           'VerifiableCredential',                                                                 
                           'VerifiableAttestation',                                                                
                           'VerifiableAccreditation',                                                              
                           'VerifiableAccreditationToAttest'                                                       
                       ],                                                                                          
                       'trust_framework':

           Authorization server configuration:                                                     ]8;id=127241;file:///tmp/ipykernel_54329/1703696794.py\1703696794.py]8;;\:]8;id=592835;file:///tmp/ipykernel_54329/1703696794.py#24\24]8;;\
           OpenIDAuthServerConfig(                                                                                 
               redirect_uris=[                                                                                     
                   'https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                          
               ],                                                                                                  
               issuer='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                                  
               authorization_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/au                 
           thorize',                                                                                               
               token_endpoint='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/token',                    
               jwks_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/jwks',                           
               scopes_supported=['openid'],                                                                        
               response_types_supported=['vp_token', 'id_token'],                                                  
               response_modes_supported=['query'],                                                                 
               grant_types_supported=['authorization_code'],                                                       
               subject_types_supported=['public'],                                                                 
               id_token_signing_alg_values_supported=['ES256'],                                                    
               request_object_signing_alg_values_supported=['ES256'],                                              
               request_parameter_supported=True,                                                                   
               request_uri_parameter_supported=True,                                                               
               token_endpoint_auth_methods_supported=['private_key_jwt'],                                          
               request_authentication_methods_supported={                                                          
                   'authorization_endpoint': ['request_object']                                                    
               },                                                                                                  
               vp_formats_supported={                                                                              
                   'jwt_vp': {'alg_values_supported': ['ES256']},                                                  
                   'jwt_vc': {'alg_values_supported': ['ES256']}                                                   
               },                                                                                                  
               subject_syntax_types_supported=['did:key', 'did:ebsi'],                                             
               subject_syntax_types_discriminations=['did:key:jwk_jcs-pub', 'did:ebsi:v1'],                        
               subject_trust_frameworks_supported=['ebsi'],                                                        
               id_token_types_supported=['subject_signed_id_token', 'attester_signed_id_token']                    
           )                                                                                                       

## 1.2 Perform authorization request and obtain ID token request

In [4]:
from ebsi_wallet.siop_auth.util import (
    perform_authorization, 
    AuthorizationRequestQueryParams,
    get_authorization_response_query_params,
    generate_code_challenge,
    generate_code_verifier
)
import uuid
import json

state = str(uuid.uuid4())
nonce = str(uuid.uuid4())
code_verifier = generate_code_verifier()

authorization_details = [{"type":"openid_credential","format":"jwt_vc","types":credential_offer.credentials[0].get("types"),"locations": ["https://api-conformance.ebsi.eu/conformance/v3/issuer-mock"]}]
redirect_uri = "http://localhost:8080"
client_metadata = {"vp_formats_supported":{"jwt_vp":{"alg":["ES256"]},"jwt_vc":{"alg":["ES256"]}},"response_types_supported":["vp_token","id_token"],"authorization_endpoint":redirect_uri}
authorization_request_query_params = AuthorizationRequestQueryParams(
    response_type="code",
    scope="openid",
    state=state,
    client_id=key_did._did,
    authorization_details=json.dumps(authorization_details, separators=(',', ':')),
    redirect_uri=redirect_uri,
    nonce=nonce,
    code_challenge=generate_code_challenge(code_verifier),
    code_challenge_method="S256",
    client_metadata=json.dumps(client_metadata, separators=(',', ':')),
    issuer_state=credential_offer.grants.get("authorization_code").get("issuer_state")
)

auth_resp = await perform_authorization(auth_server_configuration.authorization_endpoint, authorization_request_query_params)
auth_resp_uri = str(auth_resp).split("Location': '")[1].split("'")[0]

auth_resp_query_params = get_authorization_response_query_params(auth_resp_uri)
console.log("Authorization response query params: ", auth_resp_query_params)

[21:12:21] Authorization response query params:                                                    ]8;id=318748;file:///tmp/ipykernel_54329/3692576654.py\3692576654.py]8;;\:]8;id=704455;file:///tmp/ipykernel_54329/3692576654.py#36\36]8;;\
           AuthorizationResponseQueryParams(                                                                       
               state='73b59a51-51d1-4931-8e35-b166f8b685a5',                                                       
               client_id='https://api-conformance.ebsi.eu/conformance/v3/auth-mock',                               
               redirect_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/direct_post'                 
           ,                                                                                                       
               response_type='id_token',                                                                           
               response_mode='direct_post',                                                                        
               scope='openid',                                                                                     
               nonce='58d740f0-b52f-4afd-be5f-195d1264be6b',                                                       
               request_uri='https://api-conformance.ebsi.eu/conformance/v3/auth-mock/request_uri/6                 
           cecc56a-7de0-4ea2-a988-fe58472d165e'                                                                    
           )                                                                                                       

## 1.3 Send ID token

In [5]:
from ebsi_wallet.siop_auth.util import send_id_token_response

id_token = key_did.generate_id_token(auth_server_uri=auth_resp_query_params.client_id, nonce=auth_resp_query_params.nonce)
auth_code_response = await send_id_token_response(auth_resp_query_params.redirect_uri, id_token, auth_resp_query_params.state)
auth_code_response = str(auth_code_response).split("Location': '")[1].split("'")[0]
state = parse_query_string_parameters_from_url(auth_code_response).get("state")[0]
auth_code = parse_query_string_parameters_from_url(auth_code_response).get("code")[0]
console.log("Authorization code: ", auth_code)

[21:12:22] Authorization code:  d4f3a6b3-b233-4b3f-9bc2-b7ea85be3606                                ]8;id=713923;file:///tmp/ipykernel_54329/1546471797.py\1546471797.py]8;;\:]8;id=956753;file:///tmp/ipykernel_54329/1546471797.py#8\8]8;;\

## 1.4 Exchange code for access token

In [6]:
from ebsi_wallet.siop_auth.util import exchange_auth_code_for_access_token

token_uri = auth_resp_query_params.client_id + "/token"
access_token_response = await exchange_auth_code_for_access_token(token_uri, key_did._did, auth_code, code_verifier)
console.log("Access token response: ", access_token_response)

           Access token response:                                                                    ]8;id=930153;file:///tmp/ipykernel_54329/474231906.py\474231906.py]8;;\:]8;id=707285;file:///tmp/ipykernel_54329/474231906.py#5\5]8;;\
           AccessTokenResponse(                                                                                    
               access_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWV               
           ZUWlGbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6Ijg2NWY1MTA2LTlhYTYtNGUyMS1iNzU1LTViODcxN               
           jcyNWM3NyIsImNsYWltcyI6eyJhdXRob3JpemF0aW9uX2RldGFpbHMiOlt7InR5cGUiOiJvcGVuaWRfY3JlZGVudG               
           lhbCIsImZvcm1hdCI6Imp3dF92YyIsImxvY2F0aW9ucyI6WyJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV               
           1L2NvbmZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIl0sInR5cGVzIjpbIlZlcmlmaWFibGVDcmVkZW50aWFsIiwiVmVy               
           aWZpYWJsZUF0dGVzdGF0aW9uIiwiQ1RXYWxsZXRTYW1lRGVmZXJyZWQiXX1dLCJjX25vbmNlIjoiM2Y0ZDFjNTItY               
           TNjNy00NWQ2LWJlZmUtOGMxNzc0YjE0YWZlIiwiY19ub25jZV9leHBpcmVzX2luIjo4NjQwMCwiY2xpZW50X2lkIj               
           oiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JycWJaZDc0YlR               
           VOEh1aUVDQmFxR3JhYjdRcHU5UU1VbU5XZEo4R1hrcTFwNUpwV0NSWWdvZ0Z4WkxEQmtDQTF0MktjVUpoTHVwdHZv               
           R0t5Z0R4cDFNcGdFY3NpVVF0SGpOekxueFNnNTVpeGQ4MWc2d2FvZ3JmZTF0ZjJTTER0MlA2In0sImlzcyI6Imh0d               
           HBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2NrIiwiYXVkIjpbImh0dH               
           BzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvaXNzdWVyLW1vY2siXSwic3ViIjoiZGl               
           kOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JycWJaZDc0YlRVOEh1               
           aUVDQmFxR3JhYjdRcHU5UU1VbU5XZEo4R1hrcTFwNUpwV0NSWWdvZ0Z4WkxEQmtDQTF0MktjVUpoTHVwdHZvR0t5Z               
           0R4cDFNcGdFY3NpVVF0SGpOekxueFNnNTVpeGQ4MWc2d2FvZ3JmZTF0ZjJTTER0MlA2IiwiaWF0IjoxNjg3MTg5Mz               
           QyLCJleHAiOjE2ODcyNzU3NDJ9.SVc62jIB_toL1EK9Lvp3iRmQeL8eZ4t7bmr_pB7PlMXZlR4XzEabqKvPDmB6Ub               
           nCHFuS_82H8eI1ctw0GJ2xRw',                                                                              
               token_type='Bearer',                                                                                
               expires_in=86400,                                                                                   
               id_token='eyJ0eXAiOiJKV1QiLCJhbGciOiJFUzI1NiIsImtpZCI6IkJKeTViQ1Mta1dGRUhmekczYWVZUWl               
           GbmVOYnFDWHRkLVlaVmk3cmxpRGsifQ.eyJub25jZSI6IjQ1MTYyZGFkLWFjYzQtNDIzMy05MjQyLTc4N2RjNTEyN               
           2QxYSIsImlzcyI6Imh0dHBzOi8vYXBpLWNvbmZvcm1hbmNlLmVic2kuZXUvY29uZm9ybWFuY2UvdjMvYXV0aC1tb2               
           NrIiwic3ViIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2J               
           ycWJaZDc0YlRVOEh1aUVDQmFxR3JhYjdRcHU5UU1VbU5XZEo4R1hrcTFwNUpwV0NSWWdvZ0Z4WkxEQmtDQTF0Mktj               
           VUpoTHVwdHZvR0t5Z0R4cDFNcGdFY3NpVVF0SGpOekxueFNnNTVpeGQ4MWc2d2FvZ3JmZTF0ZjJTTER0MlA2IiwiY               
           XVkIjoiZGlkOmtleTp6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JycWJaZD               
           c0YlRVOEh1aUVDQmFxR3JhYjdRcHU5UU1VbU5XZEo4R1hrcTFwNUpwV0NSWWdvZ0Z4WkxEQmtDQTF0MktjVUpoTHV               
           wdHZvR0t5Z0R4cDFNcGdFY3NpVVF0SGpOekxueFNnNTVpeGQ4MWc2d2FvZ3JmZTF0ZjJTTER0MlA2IiwiaWF0Ijox               
           Njg3MTg5MzQyLCJleHAiOjE2ODcxODk0NjJ9.8NN34JRh3XPo_Ptg9gAmhDU1EGgxQQY2Ha55nNaOLTlOiwiScBQj               
           RaGrrrMqBlAAskaxfFqTQHK8B8zTwW1pVQ',                                                                    
               c_nonce='3f4d1c52-a3c7-45d6-befe-8c1774b14afe',                                                     
               c_nonce_expires_in=86400     

## 1.5 Request credential to get acceptance token to initiate deferred flow 

In [7]:
from ebsi_wallet.siop_auth.util import send_credential_request

credential_request_jwt = key_did.generate_credential_request(credential_issuer_configuration.credential_issuer, access_token_response.c_nonce)
console.log("Credential request JWT: ", credential_request_jwt)
acceptance_token_response = await send_credential_request(credential_issuer_configuration.credential_endpoint, access_token_response.access_token, credential_request_jwt, credential_offer.credentials[0].get("types"))
console.log("Acceptance token response: ", acceptance_token_response)

           Credential request JWT:                                                                  ]8;id=41588;file:///tmp/ipykernel_54329/2195976816.py\2195976816.py]8;;\:]8;id=910924;file:///tmp/ipykernel_54329/2195976816.py#4\4]8;;\
           eyJhbGciOiJFUzI1NiIsImtpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtV                
           WjNleXFodDFqOUticnFiWmQ3NGJUVThIdWlFQ0JhcUdyYWI3UXB1OVFNVW1OV2RKOEdYa3ExcDVKcFdDUllnb2dG                
           eFpMREJrQ0ExdDJLY1VKaEx1cHR2b0dLeWdEeHAxTXBnRWNzaVVRdEhqTnpMbnhTZzU1aXhkODFnNndhb2dyZmUx                
           dGYyU0xEdDJQNiN6MmRtekQ4MWNnUHg4VmtpN0pidXVNbUZZcldQZ1lveXR5a1VaM2V5cWh0MWo5S2JycWJaZDc0                
           YlRVOEh1aUVDQmFxR3JhYjdRcHU5UU1VbU5XZEo4R1hrcTFwNUpwV0NSWWdvZ0Z4WkxEQmtDQTF0MktjVUpoTHVw                
           dHZvR0t5Z0R4cDFNcGdFY3NpVVF0SGpOekxueFNnNTVpeGQ4MWc2d2FvZ3JmZTF0ZjJTTER0MlA2IiwidHlwIjoi                
           b3BlbmlkNHZjaS1wcm9vZitqd3QifQ.eyJhdWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L2Nvb                
           mZvcm1hbmNlL3YzL2lzc3Vlci1tb2NrIiwiZXhwIjoxNjg3Mjc1NzQyLCJpYXQiOjE2ODcxODkzNDIsImlzcyI6I                
           mRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticnFiWmQ3NGJUV                
           ThIdWlFQ0JhcUdyYWI3UXB1OVFNVW1OV2RKOEdYa3ExcDVKcFdDUllnb2dGeFpMREJrQ0ExdDJLY1VKaEx1cHR2b                
           0dLeWdEeHAxTXBnRWNzaVVRdEhqTnpMbnhTZzU1aXhkODFnNndhb2dyZmUxdGYyU0xEdDJQNiIsIm5vbmNlIjoiM                
           2Y0ZDFjNTItYTNjNy00NWQ2LWJlZmUtOGMxNzc0YjE0YWZlIn0.ab7jod49sZD85wIFl2jLOh7xfLBiAs5cNwX3_                
           pvLjiHbgp_nV-LdPWW2DBTTlfdETDIjibe4m23u4wiK7GfLBg                                                       

[21:12:24] Acceptance token response:                                                               ]8;id=510277;file:///tmp/ipykernel_54329/2195976816.py\2195976816.py]8;;\:]8;id=473548;file:///tmp/ipykernel_54329/2195976816.py#6\6]8;;\
           {'acceptance_token': 'ZXYWuMd2e1g7QyKMT4-Kibg9Yma9jbewuRBvO_IQi0I'}                                     

## 1.6 Request credential using deferred endpoint

In [8]:
from ebsi_wallet.siop_auth.util import send_deferred_credential_request
from time import sleep
import aiohttp
console.log("Waiting for 5 seconds...")
sleep(5)
console.log("Sending deferred credential request...")
credential_response = await send_deferred_credential_request(credential_issuer_configuration.deferred_credential_endpoint, acceptance_token_response.get("acceptance_token"))
console.log("Credential response: ", credential_response)

           Waiting for 5 seconds...                                                                 ]8;id=895804;file:///tmp/ipykernel_54329/2370662575.py\2370662575.py]8;;\:]8;id=991544;file:///tmp/ipykernel_54329/2370662575.py#4\4]8;;\

[21:12:29] Sending deferred credential request...                                                   ]8;id=519023;file:///tmp/ipykernel_54329/2370662575.py\2370662575.py]8;;\:]8;id=332656;file:///tmp/ipykernel_54329/2370662575.py#6\6]8;;\

[21:12:30] Credential response:                                                                     ]8;id=917244;file:///tmp/ipykernel_54329/2370662575.py\2370662575.py]8;;\:]8;id=887692;file:///tmp/ipykernel_54329/2370662575.py#8\8]8;;\
           {                                                                                                       
               'format': 'jwt_vc',                                                                                 
               'credential':                                                                                       
           'eyJhbGciOiJFUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImRpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWl                
           kI3VPa3Y1TE9pbzlYQnE2a1JRaHNaMHpOTVZ1czBjNFZqcFB0cDY1dTVjWXcifQ.eyJqdGkiOiJ2YzplYnNpOmNv                
           bmZvcm1hbmNlIzhkMGIwMjFjLTY5YTAtNGM3NC1hZTYzLTdkOTU0OGFhYmZlNCIsInN1YiI6ImRpZDprZXk6ejJk                
           bXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZb3l0eWtVWjNleXFodDFqOUticnFiWmQ3NGJUVThIdWlFQ0JhcUdy                
           YWI3UXB1OVFNVW1OV2RKOEdYa3ExcDVKcFdDUllnb2dGeFpMREJrQ0ExdDJLY1VKaEx1cHR2b0dLeWdEeHAxTXBn                
           RWNzaVVRdEhqTnpMbnhTZzU1aXhkODFnNndhb2dyZmUxdGYyU0xEdDJQNiIsImlzcyI6ImRpZDplYnNpOnpoSkFS                
           alBONjljRXRnUHhIZW4xTWlkIiwibmJmIjoxNjg3MTg5MzQzLCJleHAiOjE2ODcyNzU3NDMsImlhdCI6MTY4NzE4                
           OTM0MywidmMiOnsiQGNvbnRleHQiOlsiaHR0cHM6Ly93d3cudzMub3JnLzIwMTgvY3JlZGVudGlhbHMvdjEiXSwi                
           aWQiOiJ2YzplYnNpOmNvbmZvcm1hbmNlIzhkMGIwMjFjLTY5YTAtNGM3NC1hZTYzLTdkOTU0OGFhYmZlNCIsInR5                
           cGUiOlsiVmVyaWZpYWJsZUNyZWRlbnRpYWwiLCJWZXJpZmlhYmxlQXR0ZXN0YXRpb24iLCJDVFdhbGxldFNhbWVE                
           ZWZlcnJlZCJdLCJpc3N1ZXIiOiJkaWQ6ZWJzaTp6aEpBUmpQTjY5Y0V0Z1B4SGVuMU1pZCIsImlzc3VhbmNlRGF0                
           ZSI6IjIwMjMtMDYtMTlUMTU6NDI6MjNaIiwiaXNzdWVkIjoiMjAyMy0wNi0xOVQxNTo0MjoyM1oiLCJ2YWxpZEZy                
           b20iOiIyMDIzLTA2LTE5VDE1OjQyOjIzWiIsImV4cGlyYXRpb25EYXRlIjoiMjAyMy0wNi0yMFQxNTo0MjoyM1oi                
           LCJjcmVkZW50aWFsU3ViamVjdCI6eyJpZCI6ImRpZDprZXk6ejJkbXpEODFjZ1B4OFZraTdKYnV1TW1GWXJXUGdZ                
           b3l0eWtVWjNleXFodDFqOUticnFiWmQ3NGJUVThIdWlFQ0JhcUdyYWI3UXB1OVFNVW1OV2RKOEdYa3ExcDVKcFdD                
           Ullnb2dGeFpMREJrQ0ExdDJLY1VKaEx1cHR2b0dLeWdEeHAxTXBnRWNzaVVRdEhqTnpMbnhTZzU1aXhkODFnNndh                
           b2dyZmUxdGYyU0xEdDJQNiJ9LCJjcmVkZW50aWFsU2NoZW1hIjp7ImlkIjoiaHR0cHM6Ly9hcGktY29uZm9ybWFu                
           Y2UuZWJzaS5ldS90cnVzdGVkLXNjaGVtYXMtcmVnaXN0cnkvdjIvc2NoZW1hcy96M01nVUZVa2I3MjJ1cTR4M2R2                
           NXlBSm1uTm16REZlSzVVQzh4ODNRb2VMSk0iLCJ0eXBlIjoiRnVsbEpzb25TY2hlbWFWYWxpZGF0b3IyMDIxIn0s                
           InRlcm1zT2ZVc2UiOnsiaWQiOiJodHRwczovL2FwaS1jb25mb3JtYW5jZS5lYnNpLmV1L3RydXN0ZWQtaXNzdWVy                
           cy1yZWdpc3RyeS92NC9pc3N1ZXJzL2RpZDplYnNpOnpoSkFSalBONjljRXRnUHhIZW4xTWlkL2F0dHJpYnV0ZXMv                
           ZjFhY2IzNzVmY2ZiZTExYTYwMDNkNWI3MTBiZjM3ZWNiMjc4MmMxMWE0ZTQyODU3MDEzMzc3MzJjMjY2NGQwMCIs                
           InR5cGUiOiJJc3N1YW5jZUNlcnRpZmljYXRlIn19fQ.0DCCgyHryi3DIpGl00Ubcf-dQCGMN8ftC8sDPi7CjjCEJ                
           PsddpYhrHDNTfqI8gA-8krFUCuf8Kgp6Xuhaj3cng'                                                              
           }                                                                                                       